In [10]:
import pandas as pd

positive_filtered = pd.read_csv("positive_filtered.tsv" , sep="\t")
negative_filtered = pd.read_csv("negative_filtered.tsv" , sep="\t")
positive_filtered['Label']="Positive"
negative_filtered['Label']="Negative"
positive_filtered.head()

,EntryID,OrganismName,Kingdom,SequenceLength,SPStart,SPEnd,Label
0,O43155,Homo sapiens,Metazoa,660,1,35,Positive
1,O43866,Homo sapiens,Metazoa,347,1,19,Positive
2,O75629,Homo sapiens,Metazoa,220,1,31,Positive
3,O94985,Homo sapiens,Metazoa,981,1,28,Positive
4,P01579,Homo sapiens,Metazoa,166,1,23,Positive


In [11]:
from sklearn.utils import shuffle

positive_filtered_random = shuffle(positive_filtered, random_state=42).reset_index(drop=True)
negative_filtered_random = shuffle(negative_filtered, random_state=42).reset_index(drop=True)


In [12]:
from sklearn.model_selection import train_test_split

X_positive_train, X_positive_test= train_test_split(positive_filtered_random, test_size=0.2, random_state=42)
X_negative_train, X_negative_test= train_test_split(negative_filtered_random, test_size=0.2, random_state=42)


In [13]:
X_positive_train = X_positive_train.reset_index(drop=True)
X_positive_test = X_positive_test.reset_index(drop=True)
X_negative_train = X_negative_train.reset_index(drop=True)
X_negative_test = X_negative_test.reset_index(drop=True)

In [14]:
X_train = pd.concat([X_positive_train , X_negative_train], axis=0)
X_test = pd.concat([X_positive_test, X_negative_test], axis=0)

In [15]:
X_test

,EntryID,OrganismName,Kingdom,SequenceLength,SPStart,SPEnd,Label,HelixDomain
0,P51654,Homo sapiens,Metazoa,580,1.0,24.0,Positive,NaN
1,P10562,Canavalia gladiata,Viridiplantae,445,1.0,26.0,Positive,NaN
2,O35806,Rattus norvegicus,Metazoa,1764,1.0,35.0,Positive,NaN
3,P01189,Homo sapiens,Metazoa,267,1.0,26.0,Positive,NaN
4,Q9NR16,Homo sapiens,Metazoa,1453,1.0,40.0,Positive,NaN
...,...,...,...,...,...,...,...,...
1782,Q8VY52,Arabidopsis thaliana,Viridiplantae,232,NaN,NaN,Negative,False
1783,P22798,Spinacia oleracea,Viridiplantae,271,NaN,NaN,Negative,False
1784,Q24143,Drosophila melanogaster,Metazoa,723,NaN,NaN,Negative,False
1785,Q99417,Homo sapiens,Metazoa,103,NaN,NaN,Negative,False


In [16]:
!pip install biopython

In [17]:
from Bio import SeqIO
import subprocess
import shlex

In [18]:
neg_sequences = SeqIO.parse("negative_cluster_rep_seq.fasta", "fasta")
pos_sequences = SeqIO.parse("positive_cluster_rep_seq.fasta", "fasta")

all_sequences = list(neg_sequences) + list(pos_sequences)

with open("repressive_dataset.fasta", "w") as output_handle:
    SeqIO.write(all_sequences, output_handle, "fasta")


X_test['Set']="Benchmark"
rep_dataset='repressive_dataset.fasta'
ids=X_test['EntryID'].tolist()


with open(rep_dataset , 'r') as file:
    for protein in SeqIO.parse(file , 'fasta'):
        if str(protein.id) in ids:
            X_test.loc[X_test['EntryID'] == str(protein.id), 'Sequence'] = str(protein.seq)
        else:
            pass


In [19]:
X_test

,EntryID,OrganismName,Kingdom,SequenceLength,SPStart,SPEnd,Label,HelixDomain,Set,Sequence
0,P51654,Homo sapiens,Metazoa,580,1.0,24.0,Positive,NaN,Benchmark,MAGTVRTACLVVAMLLSLDFPGQAQPPPPPPDATCHQVRSFFQRLQ...
1,P10562,Canavalia gladiata,Viridiplantae,445,1.0,26.0,Positive,NaN,Benchmark,MAFSARFPLWLLLGVVLLASVSASFAHSGHSGGEAEDESEESRAQN...
2,O35806,Rattus norvegicus,Metazoa,1764,1.0,35.0,Positive,NaN,Benchmark,MRAPTTVRCSGRIQRARWRGFLPLVLALLMGTSHAQRDSVGRYEPA...
3,P01189,Homo sapiens,Metazoa,267,1.0,26.0,Positive,NaN,Benchmark,MPRSCCSRSGALLLALLLQASMEVRGWCLESSQCQDLTTESNLLEC...
4,Q9NR16,Homo sapiens,Metazoa,1453,1.0,40.0,Positive,NaN,Benchmark,MMLPQNSWHIDFGRCCCHQNLFSAVVTCILLLNSCFLISSFNGTDL...
...,...,...,...,...,...,...,...,...,...,...
1782,Q8VY52,Arabidopsis thaliana,Viridiplantae,232,NaN,NaN,Negative,False,Benchmark,MWSQSFLGSAPKLCLFSSSLPPFSHHKIHKFFCFAQNPSSTVSINL...
1783,P22798,Spinacia oleracea,Viridiplantae,271,NaN,NaN,Negative,False,Benchmark,MASLLSLSSTPPSTANSNNYPSSTFKGNINNFRINPFNFAPLKLHL...
1784,Q24143,Drosophila melanogaster,Metazoa,723,NaN,NaN,Negative,False,Benchmark,MSPPKNCAVCGDKALGYNFNAVTCESCKAFFRRNALAKKQFTCPFN...
1785,Q99417,Homo sapiens,Metazoa,103,NaN,NaN,Negative,False,Benchmark,MAHYKAADSKREQFRRYLEKSGVLDTLTKVLVALYEEPEKPNSALD...


In [20]:
ids=X_train['EntryID'].tolist()


with open(rep_dataset , 'r') as file:
    for protein in SeqIO.parse(file , 'fasta'):
        if str(protein.id) in ids:
            X_train.loc[X_train['EntryID'] == str(protein.id), 'Sequence'] = str(protein.seq)
        else:
            pass

In [21]:
X_train

,EntryID,OrganismName,Kingdom,SequenceLength,SPStart,SPEnd,Label,HelixDomain,Sequence
0,Q16552,Homo sapiens,Metazoa,155,1.0,23.0,Positive,NaN,MTPGKTSLVSLLLLLSLEAIVKAGITIPRNPGCPNSEDKNFPRTVM...
1,Q9M373,Arabidopsis thaliana,Viridiplantae,74,1.0,26.0,Positive,NaN,MASRNSVAVIALFAFVFAVISPFAGAQSLAPAPSPTSDGTSIDQGI...
2,P62520,Chilobrachys guangxiensis,Metazoa,63,1.0,27.0,Positive,NaN,MKNTSILFILGLALLLVLAFEAQVGESDGECGGFWWKCGRGKPPCC...
3,Q8I7X1,Porcellio scaber,Metazoa,145,1.0,21.0,Positive,NaN,MKGLLFIVSLLCLTLHQRVWAYQVIGMKSDVICADIRFTVHCICNE...
4,P14625,Homo sapiens,Metazoa,803,1.0,21.0,Positive,NaN,MRALWVLGLCCVLLTFGSVRADDEVDVDGTVEEDLGKSREGSRTDD...
...,...,...,...,...,...,...,...,...,...
7142,B7WN96,Caenorhabditis elegans,Metazoa,317,NaN,NaN,Negative,False,METANYYLPSPPYSSTSSSDSRESRMNTPIPTTYSEENVNSLFHLM...
7143,P42838,Saccharomyces cerevisiae (strain ATCC 204508 /...,Fungi,414,NaN,NaN,Negative,True,MVNFDLGQVGEVFRRKDKGAIVSGDNPEEEEDVDASEFEEDEVKPV...
7144,Q96MF7,Homo sapiens,Metazoa,247,NaN,NaN,Negative,False,MPGRSSSNSGSTGFISFSGVESALSSLKNFQACINSGMDTASSVAL...
7145,Q9UJS0,Homo sapiens,Metazoa,675,NaN,NaN,Negative,False,MAAAKVALTKRADPAELRTIFLKYASIEKNGEFFMSPNDFVTRYLN...


In [22]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)



X_train = X_train.reset_index(drop=True)

for fold, (_, val_idx) in enumerate(skf.split(X_train, X_train["Label"])):
    X_train.loc[val_idx, "Set"] = fold


In [23]:
X_test

,EntryID,OrganismName,Kingdom,SequenceLength,SPStart,SPEnd,Label,HelixDomain,Set,Sequence
0,P51654,Homo sapiens,Metazoa,580,1.0,24.0,Positive,NaN,Benchmark,MAGTVRTACLVVAMLLSLDFPGQAQPPPPPPDATCHQVRSFFQRLQ...
1,P10562,Canavalia gladiata,Viridiplantae,445,1.0,26.0,Positive,NaN,Benchmark,MAFSARFPLWLLLGVVLLASVSASFAHSGHSGGEAEDESEESRAQN...
2,O35806,Rattus norvegicus,Metazoa,1764,1.0,35.0,Positive,NaN,Benchmark,MRAPTTVRCSGRIQRARWRGFLPLVLALLMGTSHAQRDSVGRYEPA...
3,P01189,Homo sapiens,Metazoa,267,1.0,26.0,Positive,NaN,Benchmark,MPRSCCSRSGALLLALLLQASMEVRGWCLESSQCQDLTTESNLLEC...
4,Q9NR16,Homo sapiens,Metazoa,1453,1.0,40.0,Positive,NaN,Benchmark,MMLPQNSWHIDFGRCCCHQNLFSAVVTCILLLNSCFLISSFNGTDL...
...,...,...,...,...,...,...,...,...,...,...
1782,Q8VY52,Arabidopsis thaliana,Viridiplantae,232,NaN,NaN,Negative,False,Benchmark,MWSQSFLGSAPKLCLFSSSLPPFSHHKIHKFFCFAQNPSSTVSINL...
1783,P22798,Spinacia oleracea,Viridiplantae,271,NaN,NaN,Negative,False,Benchmark,MASLLSLSSTPPSTANSNNYPSSTFKGNINNFRINPFNFAPLKLHL...
1784,Q24143,Drosophila melanogaster,Metazoa,723,NaN,NaN,Negative,False,Benchmark,MSPPKNCAVCGDKALGYNFNAVTCESCKAFFRRNALAKKQFTCPFN...
1785,Q99417,Homo sapiens,Metazoa,103,NaN,NaN,Negative,False,Benchmark,MAHYKAADSKREQFRRYLEKSGVLDTLTKVLVALYEEPEKPNSALD...


In [24]:
X_train

,EntryID,OrganismName,Kingdom,SequenceLength,SPStart,SPEnd,Label,HelixDomain,Sequence,Set
0,Q16552,Homo sapiens,Metazoa,155,1.0,23.0,Positive,NaN,MTPGKTSLVSLLLLLSLEAIVKAGITIPRNPGCPNSEDKNFPRTVM...,4.0
1,Q9M373,Arabidopsis thaliana,Viridiplantae,74,1.0,26.0,Positive,NaN,MASRNSVAVIALFAFVFAVISPFAGAQSLAPAPSPTSDGTSIDQGI...,1.0
2,P62520,Chilobrachys guangxiensis,Metazoa,63,1.0,27.0,Positive,NaN,MKNTSILFILGLALLLVLAFEAQVGESDGECGGFWWKCGRGKPPCC...,2.0
3,Q8I7X1,Porcellio scaber,Metazoa,145,1.0,21.0,Positive,NaN,MKGLLFIVSLLCLTLHQRVWAYQVIGMKSDVICADIRFTVHCICNE...,4.0
4,P14625,Homo sapiens,Metazoa,803,1.0,21.0,Positive,NaN,MRALWVLGLCCVLLTFGSVRADDEVDVDGTVEEDLGKSREGSRTDD...,2.0
...,...,...,...,...,...,...,...,...,...,...
8016,B7WN96,Caenorhabditis elegans,Metazoa,317,NaN,NaN,Negative,False,METANYYLPSPPYSSTSSSDSRESRMNTPIPTTYSEENVNSLFHLM...,4.0
8017,P42838,Saccharomyces cerevisiae (strain ATCC 204508 /...,Fungi,414,NaN,NaN,Negative,True,MVNFDLGQVGEVFRRKDKGAIVSGDNPEEEEDVDASEFEEDEVKPV...,1.0
8018,Q96MF7,Homo sapiens,Metazoa,247,NaN,NaN,Negative,False,MPGRSSSNSGSTGFISFSGVESALSSLKNFQACINSGMDTASSVAL...,0.0
8019,Q9UJS0,Homo sapiens,Metazoa,675,NaN,NaN,Negative,False,MAAAKVALTKRADPAELRTIFLKYASIEKNGEFFMSPNDFVTRYLN...,1.0


In [25]:
data = pd.concat([X_train , X_test] , ignore_index=True)
data.to_csv("data.tsv",sep="\t", index=False)

In [26]:
print("Training Positives",len(X_train.loc[X_train['Label'] == "Positive"]))
print("Training Negatives",len(X_train.loc[X_train['Label'] == "Negative"]))
print("Benchmark Positives",len(X_test.loc[X_test['Label'] == "Positive"]))
print("Benchmark Positives",len(X_test.loc[X_test['Label'] == "Negative"]))

Training Positives 874
Training Negatives 7147
Benchmark Positives 219
Benchmark Positives 1787
